---
title: "Benchmarking: Projection"
---

In [1]:
import hvplot
import holoviews as hv
import pandas as pd
import hvplot.pandas  # noqa

import statsmodels.formula.api as smf

pd.options.plotting.backend = "holoviews"

Read summary of all benchmarking results.

In [2]:
summary = pd.read_parquet("s3://carbonplan-benchmarks/benchmark-data/v0.2/summary.parq")

Subset the data to isolate the impact of projection and chunk size.

In [3]:
df = summary[(summary["pixels_per_tile"] == 128) & (summary["region"] == "us-west-2")]

Set plot options.

In [4]:
cmap = ["#E1BE6A", "#40B0A6"]
plt_opts = {"width": 600, "height": 400}

Create a box plot showing how the rendering time depends on Zarr version and chunk size.

In [5]:
df.hvplot.box(
    y="duration",
    by=["actual_chunk_size", "projection"],
    c="projection",
    cmap=cmap,
    ylabel="Action duration (ms)",
    xlabel="Chunk size (MB); EPSG number",
    legend=False,
).opts(**plt_opts)

:BoxWhisker   [actual_chunk_size,projection]   (duration)

Fit a multiple linear regression to the results. The results show that the projection, along with the chunk size, strongly impacts the rendering time, with web mercator pyramids rendering faster than equidistant cylindrical pyramids.

In [6]:
model = smf.ols("duration ~ actual_chunk_size + C(projection)", data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.430
Method:                 Least Squares   F-statistic:                     387.2
Date:                Tue, 29 Aug 2023   Prob (F-statistic):          7.16e-126
Time:                        17:35:31   Log-Likelihood:                -8152.5
No. Observations:                1024   AIC:                         1.631e+04
Df Residuals:                    1021   BIC:                         1.633e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              1864.6662     39.029     47.776      0.000    1788.080    1941.252
C(projection)[T.4326]   580.6684     43.438     13.368      0.000     495.430     665.906
actual_chunk_size        60.3908      2.474     24.408      0.000      55.536      65.246
==============================================================================
Omnibus:                        7.834   Durbin-Watson:                   1.830
Prob(Omnibus):                  0.020   Jarque-Bera (JB):                7.973
Skew:                          -0.210   Prob(JB):                       0.0186
Kurtosis:                       2.897   Cond. No.                         31.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Show the rendering time at different zoom levels.

In [7]:
plt_opts = {"width": 400, "height": 300}

plts = []

for zoom_level in range(4):
    df_level = df[df["zoom"] == zoom_level]
    plts.append(
        df_level.hvplot.box(
            y="duration",
            by=["actual_chunk_size", "projection"],
            c="projection",
            cmap=cmap,
            ylabel="Action duration (ms)",
            xlabel="Chunk size (MB); EPSG number",
            legend=False,
            title=f"Zoom level {zoom_level}",
        ).opts(**plt_opts)
    )
hv.Layout(plts).cols(2)

/Users/max/mambaforge/envs/benchmark-maps/lib/python3.10/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/Users/max/mambaforge/envs/benchmark-maps/lib/python3.10/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


:Layout
   .BoxWhisker.I   :BoxWhisker   [actual_chunk_size,projection]   (duration)
   .BoxWhisker.II  :BoxWhisker   [actual_chunk_size,projection]   (duration)
   .BoxWhisker.III :BoxWhisker   [actual_chunk_size,projection]   (duration)
   .BoxWhisker.IV  :BoxWhisker   [actual_chunk_size,projection]   (duration)

Add a multiplicative interaction term with zoom level to the multiple linear regression. The results show that projection has a significant impact on rendering performance at higher zoom levels, with most pronounced affect at zoom level 3.

In [8]:
model = smf.ols("duration ~ actual_chunk_size + C(projection) * C(zoom)", data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     297.4
Date:                Tue, 29 Aug 2023   Prob (F-statistic):          6.60e-260
Time:                        17:35:31   Log-Likelihood:                -7823.4
No. Observations:                1024   AIC:                         1.566e+04
Df Residuals:                    1015   BIC:                         1.571e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                             1590.6019     47.989     33.145      0.000    1496.433    1684.771
C(projection)[T.4326]                   27.3709     63.186      0.433      0.665     -96.619     151.361
C(zoom)[T.1.0]                        1043.0435     63.186     16.507      0.000     919.053    1167.034
C(zoom)[T.2.0]                         239.5532     63.186      3.791      0.000     115.563     363.543
C(zoom)[T.3.0]                        -186.3396     63.186     -2.949      0.003    -310.330     -62.349
C(projection)[T.4326]:C(zoom)[T.1.0]    42.7015     89.359      0.478      0.633    -132.647     218.050
C(projection)[T.4326]:C(zoom)[T.2.0]   741.8614     89.359      8.302      0.000     566.513     917.210
C(projection)[T.4326]:C(zoom)[T.3.0]  1428.6270     89.359     15.988      0.000    1253.278    1603.976
actual_chunk_size                       60.3908      1.800     33.559      0.000      56.860      63.922
==============================================================================
Omnibus:                       62.427   Durbin-Watson:                   1.316
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               73.083
Skew:                          -0.612   Prob(JB):                     1.35e-16
Kurtosis:                       3.463   Cond. No.                         132.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""